In [56]:
!pip install snowflake-sqlalchemy # Install the 'snowflake-sqlalchemy' package
!pip install snowflake-connector-python

from sqlalchemy import create_engine
from snowflake.sqlalchemy import URL

# Your Snowflake credentials
credenciales = {
    'account': 'EMEOOVH-UQ32135',  # Replace with your account identifier
    'user': 'APLEMAE',      # Replace with your username
    'password': 'Aple1234567890',  # Replace with your password
    'database': 'PARTIDOS_DB',   # Replace with your database name
    'schema': 'ROW'  # Add if you need to specify a schema
}

# Create the SQLAlchemy engine
engine = create_engine(URL(
    account=credenciales['account'],
    user=credenciales['user'],
    password=credenciales['password'],
    database=credenciales['database'],
    schema=credenciales['schema'] # Add if you specified a schema in 'credenciales'
))

# Test the connection
try:
    with engine.connect() as connection:
        print("¡Conexión exitosa!")
except Exception as e:
    print(f"Error de conexión: {str(e)}")

ERROR: Invalid requirement: '#': Expected package name at the start of dependency specifier
    #
    ^


¡Conexión exitosa!


In [57]:

import pandas as pd

# Use double quotes to escape the schema name "ROW"
consulta='select * from PARTIDOS_DB."RAW".RESULTADOS'
raw_partidos=pd.read_sql(consulta,engine)


In [58]:
#date viene como string
#Transformación de fechas a tipo de dato fechatiempo
#raw_partidos ya queda con la fecha ok

raw_partidos['date']=pd.to_datetime(raw_partidos['date'])
raw_partidos['mes']=raw_partidos['date'].dt.month
raw_partidos['año']=raw_partidos['date'].dt.year

In [60]:
raw_partidos.head()

,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral,mes,año
0,1872-11-30,Scotland,England,0,0,Friendly,Glasgow,Scotland,False,11,1872
1,1873-03-08,England,Scotland,4,2,Friendly,London,England,False,3,1873
2,1874-03-07,Scotland,England,2,1,Friendly,Glasgow,Scotland,False,3,1874
3,1875-03-06,England,Scotland,2,2,Friendly,London,England,False,3,1875
4,1876-03-04,Scotland,England,3,0,Friendly,Glasgow,Scotland,False,3,1876


In [64]:
# Crear resultado
def get_resultado(row):
    if row['home_team'] == 'Ecuador':
        if row['home_score'] > row['away_score']:
            return 'gana'
        elif row['home_score'] < row['away_score']:
            return 'pierde'
        else:
            return 'empata'
    elif row['away_team'] == 'Ecuador':
        if row['away_score'] > row['home_score']:
            return 'gana'
        elif row['away_score'] < row['home_score']:
            return 'pierde'
        else:
            return 'empata'
    else:
        return 'pierde' #Quien no jugó con Ecuador queda en pierde

raw_partidos['resultado'] = raw_partidos.apply(get_resultado, axis=1)

# Filtra Ecuador
df_ecuador = raw_partidos[(raw_partidos['home_team'] == 'Ecuador') | (raw_partidos['away_team'] == 'Ecuador')].copy()

# Create the 'oponente' column
df_ecuador.loc[df_ecuador['home_team'] == 'Ecuador', 'oponente'] = df_ecuador['away_team']
df_ecuador.loc[df_ecuador['away_team'] == 'Ecuador', 'oponente'] = df_ecuador['home_team']

# Explore the new dataframe
display(df_ecuador.head())
print(df_ecuador.shape)
print(df_ecuador['resultado'].value_counts())

,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral,mes,año,resultado,oponente
2373,1938-08-08,Bolivia,Ecuador,1,1,Bolivarian Games,Bogotá,Colombia,True,8,1938,empata,Bolivia
2375,1938-08-10,Colombia,Ecuador,1,2,Bolivarian Games,Bogotá,Colombia,False,8,1938,gana,Colombia
2377,1938-08-11,Ecuador,Peru,1,9,Bolivarian Games,Bogotá,Colombia,True,8,1938,pierde,Peru
2385,1938-08-19,Ecuador,Venezuela,5,2,Bolivarian Games,Bogotá,Colombia,True,8,1938,gana,Venezuela
2386,1938-08-22,Bolivia,Ecuador,2,1,Bolivarian Games,Bogotá,Colombia,True,8,1938,pierde,Bolivia


(579, 13)
resultado
pierde    251
gana      179
empata    149
Name: count, dtype: int64


In [66]:
# Crear 2 columnas más de partido previo y porcentaje inicial de ganado
df_ecuador['partidos_previos'] = 0
df_ecuador['porcentaje_ganado'] = 0.5  # Porcentaje por defecto

# Calcular features
for index, row in df_ecuador.iterrows():
    oponente = row['oponente']
    fecha_actual = row['date']
    partidos_pasados = df_ecuador[(df_ecuador['oponente'] == oponente) & (df_ecuador['date'] < fecha_actual)]
    df_ecuador.loc[index, 'partidos_previos'] = len(partidos_pasados)
    ganados = len(partidos_pasados[partidos_pasados['resultado'] == 'gana'])
    if len(partidos_pasados) > 0:
        df_ecuador.loc[index, 'porcentaje_ganado'] = ganados / len(partidos_pasados)

display(df_ecuador.head())

,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral,mes,año,resultado,oponente,partidos_previos,porcentaje_ganado
2373,1938-08-08,Bolivia,Ecuador,1,1,Bolivarian Games,Bogotá,Colombia,True,8,1938,empata,Bolivia,0,0.5
2375,1938-08-10,Colombia,Ecuador,1,2,Bolivarian Games,Bogotá,Colombia,False,8,1938,gana,Colombia,0,0.5
2377,1938-08-11,Ecuador,Peru,1,9,Bolivarian Games,Bogotá,Colombia,True,8,1938,pierde,Peru,0,0.5
2385,1938-08-19,Ecuador,Venezuela,5,2,Bolivarian Games,Bogotá,Colombia,True,8,1938,gana,Venezuela,0,0.5
2386,1938-08-22,Bolivia,Ecuador,2,1,Bolivarian Games,Bogotá,Colombia,True,8,1938,pierde,Bolivia,1,0.0


In [70]:
df_ecuador

,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral,mes,año,resultado,oponente,partidos_previos,porcentaje_ganado
2373,1938-08-08,Bolivia,Ecuador,1,1,Bolivarian Games,Bogotá,Colombia,True,8,1938,empata,Bolivia,0,0.500000
2375,1938-08-10,Colombia,Ecuador,1,2,Bolivarian Games,Bogotá,Colombia,False,8,1938,gana,Colombia,0,0.500000
2377,1938-08-11,Ecuador,Peru,1,9,Bolivarian Games,Bogotá,Colombia,True,8,1938,pierde,Peru,0,0.500000
2385,1938-08-19,Ecuador,Venezuela,5,2,Bolivarian Games,Bogotá,Colombia,True,8,1938,gana,Venezuela,0,0.500000
2386,1938-08-22,Bolivia,Ecuador,2,1,Bolivarian Games,Bogotá,Colombia,True,8,1938,pierde,Bolivia,1,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47811,2024-10-15,Uruguay,Ecuador,0,0,FIFA World Cup qualification,Montevideo,Uruguay,False,10,2024,empata,Uruguay,49,0.163265
47846,2024-11-14,Ecuador,Bolivia,4,0,FIFA World Cup qualification,Guayaquil,Ecuador,False,11,2024,gana,Bolivia,37,0.513514
47951,2024-11-19,Colombia,Ecuador,0,1,FIFA World Cup qualification,Barranquilla,Colombia,False,11,2024,gana,Colombia,49,0.265306
48084,2025-03-21,Ecuador,Venezuela,2,1,FIFA World Cup qualification,Quito,Ecuador,False,3,2025,gana,Venezuela,32,0.468750


In [50]:
df_ecuador.to_csv('../DATASET/CLEAN/df_ecuador.csv', index=False)